# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core import Environment,ScriptRunConfig
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.train.automl import AutoMLConfig

import os


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_leukemia_experiment'

auto_exp = Experiment(ws,experiment_name)

In [ ]:
# Create compute cluster

cpu_cluster_name = "cluster-leukemia"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',min_nodes=0,
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

### Dataset is hosted on Github

In [4]:
dataset_path = 'https://raw.githubusercontent.com/PhilippRamjoue/Leukemia_Classification/main/dataset/bone-marrow-dataset.csv'

ds = Dataset.Tabular.from_delimited_files(path=dataset_path)

In [5]:

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.4,random_state=42)

automl_training_set = pd.concat([x_train,y_train],axis=1)

Shape of original frame: (187, 38)
Shape of frame with dropped nan rows: (164, 38); Loss: 12 %


## Save data in datastore

A datastore is necessary to run AutoML on a remote compute

In [ ]:

# Create a project_folder if it doesn't exist
if not os.path.isdir('data'):
    os.mkdir('data')

if not os.path.exists('project_folder'):
    os.makedirs('project_folder')

automl_training_set.to_csv('data/cleaned_dataset.csv')

ds = ws.get_default_datastore()

ds.upload(src_dir='./data', target_path='leukemiadata', overwrite=True, show_progress=True)

training_data = Dataset.Tabular.from_delimited_files(path=ds.path('leukemiadata/cleaned_dataset.csv'))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

,donor_age_below_35,donor_CMV,recipient_age_below_10,recipient_gender,recipient_rh,recipient_CMV,gender_match,ABO_match,HLA_mismatch,risk_group,...,allel_2,allel_3,allel_4,HLA_group_1_DRB1_cell,HLA_group_1_matched,HLA_group_1_mismatched,HLA_group_1_one_allel,HLA_group_1_one_antigen,HLA_group_1_three_diffs,HLA_group_1_two_diffs
0,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,1,0,0,0,0,0
1,1,0,1,1,1,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,1,1,1,1,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,1,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,0,1,1,1,1,1,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
160,0,1,0,1,0,1,0,1,1,1,...,0,0,0,0,1,0,0,0,0,0
161,1,0,0,0,1,1,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
162,1,1,0,0,1,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
# TODO: Put your automl settings here
automl_settings = {'experiment_timeout_minutes':30,
                    'primary_metric':'accuracy',
                    'enable_early_stopping':True,
                    'n_cross_validations':5}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="classification",
    compute_target=cpu_cluster,
    training_data=training_data,
    iterations=30,
    iteration_timeout_minutes=5,
    label_column_name='survival_status',
    **automl_settings
    )

In [15]:
# TODO: Submit your experiment
auto_run = auto_exp.submit(config=automl_config,show_output=True)

In [16]:
# TODO: Submit your experiment
auto_run = auto_exp.submit(config=automl_config,show_output=True)

Running on local machine
Parent Run ID: AutoML_eda4044b-0ed5-4549-bb34-98848a02cf27

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing f

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
RunDetails(auto_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
# Retrieve and save your best automl model.
best_run,fitted_model = auto_run.get_output(iteration=???)

In [ ]:
#Example code from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-features#scaling-and-normalization

from pprint import pprint
model = fitted_model

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(model)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.

In [ ]:
model_name = best_run.properties['model_name']
# save best AutoML model
automl_model= auto_run.register_model(model_name=model_name)



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Code from https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py#model-deploy

from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

# Register the model to deploy
model = run.register_model(model_name = "mymodel", model_path = "outputs/model.pkl")

# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "my_web_service",
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = deployment_config)

service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:


import json


input_payload = json.dumps({
    'data': dataset_x[0:2].tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)



TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()

In [ ]:
# delete compute cluster
cpu_cluster.delete()
